In [1]:
# Dataset id from Hugging Face
dataset_id = "wikitext2"

Using optimum.gptq for quantisation as runs on GPU, as opposed to transformers GPTQConfig running on CPU, much(!) faster

In [2]:
from optimum.gptq import GPTQQuantizer

# GPTQ quantizer
quantizer = GPTQQuantizer(bits=4, dataset=dataset_id, model_seqlen=2048)
quantizer.quant_method = "gptq"

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Hugging Face model id
model_id = "CorticalStack/travel-mistral-7B-GGUF"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False) # bug with fast tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, low_cpu_mem_usage=True, torch_dtype=torch.float16) # we load the model in fp16 on purpose


OSError: Can't load tokenizer for 'CorticalStack/travel-mistral-7B-GGUF'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'CorticalStack/travel-mistral-7B-GGUF' is the correct path to a directory containing all relevant files for a LlamaTokenizer tokenizer.

In [6]:
import os
import json

# quantize the model
quantized_model = quantizer.quantize_model(model, tokenizer)

# save the quantize model to disk
save_folder = "quantized_llama"
quantized_model.save_pretrained(save_folder, safe_serialization=True)

# load fresh, fast tokenizer and save it to disk
tokenizer = AutoTokenizer.from_pretrained(model_id).save_pretrained(save_folder)

# save quantize_config.json for TGI
with open(os.path.join(save_folder, "quantize_config.json"), "w", encoding="utf-8") as f:
  quantizer.disable_exllama = False
  json.dump(quantizer.to_dict(), f, indent=2)

Token indices sequence length is longer than the specified maximum sequence length for this model (2763962 > 32768). Running this sequence through the model will result in indexing errors


Quantizing model.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Found modules on cpu/disk. Using Exllama/Exllamav2 backend requires all the modules to be on GPU. Setting `disable_exllama=True`


In [ ]:
from transformers import GPTQConfig
gptq_config = GPTQConfig(bits=4, dataset="wikitext2", tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForCausalLM
quantized_model = AutoModelForCausalLM.from_pretrained('model_q4_k_m_gguf', device_map="auto", quantization_config=gptq_config)